In [ ]:
def TanhMethod(n,B):
    # inputs: 
    #   n - number of tanh functions
    #   B - the slope around the inflection point
    
    # outputs:     
    # initialize lists for the tanh function
    RightKink  = []
    LeftKink   = []
    
    # amplitude for each wave
    A          = []
    
    # maximum allowed amplitude 
    max_amplitude = 0.125
   
    # vertical shift along the y axis
    # one is chosed to keep the inflection points above 
    # zero
    S_vertical = 1
    
    # rj is the list of inflection point locations
    rj = list(linspace(1,0,n))
    r_min = 0.2
    r_max = 1
    
    # rescaling the radius (redundant but needed for BC Fairing Function)
    r_hat = (r - r_min)/(r_max - r_min)
    
    # messages for error and warning checking 
    warning_mssg = {1:'Warning: Total Amplitude exceeds maximum ', \
                    2:'Warning: Function is negative'}
    
    # getting amplitude for each kink
    for i in range(len(rj)):
        # 
        A.append(max_amplitude/(len(rj)+1))
        if sum(A) > max_amplitude:
            sys.exit(str(warning_mssg[1]))
    
    # defining kinks and antikinks
    for j in range(len(rj)):
        RightKink.append( A[j]*sp.tanh(B*( r_hat    - rj[j] )) )
        LeftKink.append(  A[j]*sp.tanh(B*( rj[j]- rj[0] )) )   
    
    f = sum(LeftKink) + sum(RightKink) + S_vertical
    return f

In [ ]:
RightKink = \
0.1*sp.tanh(0.1*(r - 1)) 
#+ \
#k[1]*sp.tanh(k[2]*(r - r2)) + \
#k[1]*sp.tanh(k[2]*(r - r3))

LeftKink = \
0.1*sp.tanh(0.1*(1 - 1)) 
#+ \
#k[1]*sp.tanh(k[2]*(r2    - r_max)) + \
#k[1]*sp.tanh(k[2]*(r3    - r_max))

S_vertical        = 1

In [ ]:
f = TanhMethod(3,100)
r_min = 0.2
r_max = 1
r_hat = (r - r_min)/(r_max - r_min)
f_min = f.subs(r,r_min)
f_max = f.subs(r,r_max)
f_min_desired = 0.00000
f_max_desired = 0.00000
A_max         = 3*r_hat**2 - 2*r_hat**3
A_min         =  1 - A_max 

 
del_f_min = abs(f_min - f_min_desired)
del_f_max = abs(f_max - f_max_desired)

f_imposed = (f + A_min*del_f_min - A_max*del_f_max)

In [ ]:
A_analytic        = S_vertical + LeftKink + RightKink 

dA_analytic_dr    = diff(A_analytic,r)
dA_analytic_sq_dr = diff(A_analytic**two,r)
M_t_analytic      = (  r/((kappa-one)*A_analytic**two) * \
                       (dA_analytic_sq_dr))**0.5


M_x_analytical = r - 1#sp.sin((r - r_max))

In [ ]:
#A_analytic =A_analytic.subs({r_max:1.0})
#,k[2]:0.2,k[1]:0.3}) 
display(A_analytic)
lam_x = lambdify(r,A_analytic , modules=['numpy'])

x_vals = linspace(r_min, r_max, 100)
y_vals = lam_x(x_vals)
debug = 0
if debug == 1:
    for i in range(len(y_vals)):
        if y_vals[i] < 0:
            sys.exit(str(warning_mssg[2]))
        
mpl.plot(x_vals, y_vals)
mpl.xlabel('radius')
mpl.show()

In [ ]:
v_t_analytical = sp.sin(k[5]*(r - r_max))
v_x_analytical = sp.sin(k[6]*(r - r_max))


# v_r and dp_dr need to be zero at the wall!
v_r_analytical = f_imposed#sp.cos(k[4]*(r - r_max)) - 1
p_analytical   = f_imposed#sp.cos(k[7]*(r - r_max)) - 1

dp_dr_analytical   = p_analytical.diff(r)
dv_r_dr_analytical = v_r_analytical.diff(r) 
dM_x_dr_analytical = M_x_analytical.diff(r)
dM_t_dr_analytical = M_t_analytic.diff(r)

In [ ]:
#A_max = 3*r**2 - 2*r**3
#A_min = 1 - A_max
#
#
#f = v_r_analytical
#f_max = sp.cos(k[4]*(r_max-r_max)) - 1
#f_min = sp.cos(k[4]*(0.2-r_max)) - 1
#f_min_desired = 0
#f_max_desired = 0
#
#del_f_min = f_min - f_min_desired
#del_f_max = f_max - f_max_desired 
#f_imposed = f + A_min*del_f_min - A_max*del_f_max
#v_r_analytical = f_imposed